# Cours : Programmation Orientée Objet et Principes SOLID en Python

## Introduction à la POO

### Pourquoi la POO ?

**Problème sans POO :**
Imaginez gérer une bibliothèque avec des dictionnaires et des fonctions dispersées.

In [ ]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(filename)s:%(lineno)d - %(name)s: %(message)s"
)

In [ ]:
# Approche sans POO
book1 = {"titre": "1984", "auteur": "Orwell", "disponible": True}
book2 = {"titre": "Dune", "auteur": "Herbert", "disponible": False}

def borrow_book(book):
    if book["disponible"]:
        book["disponible"] = False
        return True
    return False

print(f"Emprunt réussi : {borrow_book(book1)}")
print(f"Livre 1 : {book1}")

**Problèmes :**
- Pas de validation des données
- Code dupliqué partout
- Difficile à maintenir et déboguer
- Pas de cohérence entre les objets
- Impossible de tracer les modifications

## 1. Classes et Instances

Une classe est un **modèle** pour créer des objets.

In [ ]:
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
        self.available = True

book = Book('1984', 'George Orwell')
logging.info(book)

### Méthode magique : `__str__`

`__str__` définit comment un objet est affiché pour un humain (print, logs), au lieu d'une représentation technique inutile.

In [ ]:
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
        self.available = True
    
    def __str__(self):
        return f"{self.title} – {self.author}"

book = Book('1984', 'George Orwell')
logging.info(book)  # Maintenant lisible !

### Méthode magique : `__eq__`

`__eq__` définit quand deux objets doivent être considérés égaux par leur contenu, et non par leur identité.

In [ ]:
# Sans __eq__
book1 = Book('1984', 'George Orwell')
book2 = Book('1984', 'George Orwell')
logging.info(f"Les livres sont égaux ? {book1 == book2}")  # False !

In [ ]:
class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
        self.available = True
    
    def __str__(self):
        return f"{self.title} – {self.author}"

    def __eq__(self, other):
        if not isinstance(other, Book):
            return NotImplemented
        return self.title == other.title and self.author == other.author

book1 = Book('1984', 'George Orwell')
book2 = Book('1984', 'George Orwell')
logging.info(f"Les livres sont égaux ? {book1 == book2}")  # True !

## 2. Dataclasses : Simplification moderne

Les dataclasses automatisent la création de `__init__`, `__repr__`, `__eq__` et plus encore.

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class Book:
    title: str
    author: str
    available: bool = True

book1 = Book('1984', 'George Orwell')
book2 = Book('1984', 'George Orwell')

logging.info(book1)
logging.info(f"Les livres sont égaux ? {book1 == book2}")

**Avantages des dataclasses :**
- ✅ Moins de code boilerplate
- ✅ Type hints intégrés
- ✅ Valeurs par défaut simples
- ✅ Représentation automatique lisible

**On va garder dataclass pour la suite par praticité.**

## 3. Encapsulation

Protéger les données internes avec des attributs privés/protégés.

In [ ]:
class BankAccount:
    def __init__(self, initial_balance):
        self.__balance = initial_balance  # Privé (__)

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Le montant à déposer doit être positif")
        self.__balance += amount
        logging.info(f"Dépôt de {amount}€. Nouveau solde : {self.__balance}€")

    def get_balance(self):
        return self.__balance

# Test
account = BankAccount(100)
account.deposit(50)
logging.info(f"Solde : {account.get_balance()}€")

## 4. Héritage

Réutiliser du code en créant des sous-classes.

In [ ]:
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
        logging.info(f"Utilisateur créé : {name}")

class Librarian(User):
    def __init__(self, name, email, department):
        super().__init__(name, email)  # Appelle le constructeur parent
        self.department = department
        self.permissions = ["borrow", "add_book", "delete_book"]
        logging.info(f"Bibliothécaire créé dans le département : {department}")

# Test
librarian = Librarian("Marie", "marie@library.com", "Histoire")
logging.info(f"Permissions : {librarian.permissions}")

## 5. Polymorphisme

Même interface, comportements différents.

In [ ]:
class Document:
    def __init__(self, title):
        self.title = title
    
    def display(self):
        raise NotImplementedError

class Book(Document):
    def display(self):
        return f"📚 Livre: {self.title}"

class Magazine(Document):
    def display(self):
        return f"📰 Magazine: {self.title}"

# Test polymorphique
documents = [
    Book("1984"),
    Magazine("National Geographic")
]

for doc in documents:
    logging.info(doc.display())

## 6. Décorateurs Python

### @property - Propriétés gérées

In [ ]:
class Book:
    def __init__(self, title, price):
        self._title = title
        self._price = price

    @property
    def price(self):
        """Getter"""
        logging.info(f"[LOG] Accès au prix de '{self._title}'")
        return self._price

    @price.setter
    def price(self, new_price):
        """Setter avec validation"""
        if new_price < 0:
            raise ValueError("Le prix ne peut pas être négatif")
        logging.info(f"[LOG] Prix modifié: {self._price} → {new_price}")
        self._price = new_price

# Test
book = Book("1984", 15.99)
print(f"Prix initial : {book.price}€")
book.price = 12.99
print(f"Nouveau prix : {book.price}€")

### @staticmethod - Méthodes utilitaires

In [ ]:
class LibraryUtils:
    @staticmethod
    def format_isbn(isbn):
        """Formatage ISBN sans besoin de l'instance"""
        return f"{isbn[:3]}-{isbn[3:4]}-{isbn[4:9]}-{isbn[9:12]}-{isbn[12]}"

# Test
isbn = "9782070368228"
formatted = LibraryUtils.format_isbn(isbn)
logging.info(f"ISBN formaté : {formatted}")

### @classmethod - Méthodes de classe

In [ ]:
class Book:
    total_count = 0

    def __init__(self, title):
        self.title = title
        Book.total_count += 1

    @classmethod
    def from_dict(cls, data):
        """Factory method - constructeur alternatif"""
        return cls(data['title'])

    @classmethod
    def get_total_count(cls):
        return cls.total_count

# Test
book1 = Book("1984")
book2 = Book.from_dict({'title': 'Dune'})
logging.info(f"Nombre total de livres : {Book.get_total_count()}")

### @abstractmethod - Classes abstraites

In [ ]:
from abc import ABC, abstractmethod

class Borrowable(ABC):
    @abstractmethod
    def borrow(self):
        pass

    @abstractmethod
    def return_item(self):
        pass

class Book(Borrowable):
    def __init__(self, title):
        self.title = title
        self.available = True

    def borrow(self):
        if not self.available:
            raise Exception(f"Le livre '{self.title}' n'est pas disponible")
        self.available = False
        return f"Livre '{self.title}' emprunté"

    def return_item(self):
        self.available = True
        return f"Livre '{self.title}' retourné"

# Test
book = Book("1984")
logging.info(book.borrow())
logging.info(book.return_item())

## 7. Principes SOLID

### S - Single Responsibility Principle (SRP)

**Principe :** Une classe = une responsabilité.

In [ ]:
# ❌ MAUVAIS EXEMPLE
class Book:
    def __init__(self, title):
        self.title = title

    def borrow(self):
        pass

    def send_reminder_email(self):
        # PROBLÈME : Book ne devrait pas gérer les emails
        pass

    def save_to_db(self):
        # PROBLÈME : Book ne devrait pas gérer la DB
        pass

In [ ]:
# ✅ BON EXEMPLE
from dataclasses import dataclass
from datetime import datetime

@dataclass
class Book:
    """Responsabilité : Représenter un livre"""
    title: str
    author: str
    available: bool = True

@dataclass
class Borrow:
    book: Book
    user: str
    borrow_date: datetime

class BorrowManager:
    """Responsabilité : Gérer les emprunts"""
    def borrow_book(self, book, user):
        if book.available:
            book.available = False
            borrow = Borrow(book, user, datetime.now())
            logging.info(f"Emprunt créé : {book.title} par {user}")
            return borrow
        raise Exception(f"Livre non disponible : {book.title}")

class EmailService:
    """Responsabilité : Envoyer des emails"""
    def send_reminder(self, borrow):
        logging.info(f"Email envoyé pour : {borrow.book.title}")

# Test
book = Book("1984", "George Orwell")
manager = BorrowManager()
email_service = EmailService()

borrow = manager.borrow_book(book, "Alice")
email_service.send_reminder(borrow)

### O - Open/Closed Principle (OCP)

**Principe :** Ouvert à l'extension, fermé à la modification.

In [ ]:
# ❌ MAUVAIS EXEMPLE
class DiscountCalculator:
    def calculate(self, user_type, amount):
        if user_type == "student":
            return amount * 0.9
        elif user_type == "senior":
            return amount * 0.85
        elif user_type == "vip":
            return amount * 0.8
        return amount
        # Problème : Ajouter un nouveau type = modifier cette classe

In [ ]:
# ✅ BON EXEMPLE
from abc import ABC, abstractmethod

class DiscountStrategy(ABC):
    @abstractmethod
    def apply(self, amount):
        pass

class StudentDiscount(DiscountStrategy):
    def apply(self, amount):
        return amount * 0.9

class SeniorDiscount(DiscountStrategy):
    def apply(self, amount):
        return amount * 0.85

class VIPDiscount(DiscountStrategy):
    def apply(self, amount):
        return amount * 0.8

class User:
    def __init__(self, name, discount_strategy):
        self.name = name
        self.discount_strategy = discount_strategy

    def calculate_price(self, amount):
        return self.discount_strategy.apply(amount)

# Test
student = User("Alice", StudentDiscount())
senior = User("Bob", SeniorDiscount())

logging.info(f"Prix étudiant : {student.calculate_price(100)}€")
logging.info(f"Prix senior : {senior.calculate_price(100)}€")

### L - Liskov Substitution Principle (LSP)

**Principe :** Les sous-classes doivent pouvoir remplacer leurs classes parentes.

In [ ]:
# ❌ MAUVAIS EXEMPLE
class Bird:
    def fly(self):
        return "Je vole"

class Penguin(Bird):
    def fly(self):
        raise Exception("Je ne peux pas voler!")
        # PROBLÈME : Penguin ne peut pas remplacer Bird

In [ ]:
# ✅ BON EXEMPLE
class Document:
    def __init__(self, title):
        self.title = title

    def can_be_borrowed(self):
        return True

class Book(Document):
    def can_be_borrowed(self):
        return True  # Cohérent avec Document

class ArchiveDocument(Document):
    def can_be_borrowed(self):
        return False  # Cohérent, juste une réponse différente

def display_availability(document: Document):
    status = "disponible" if document.can_be_borrowed() else "archivé"
    logging.info(f"{document.title}: {status}")

# Test
book = Book("1984")
archive = ArchiveDocument("Manuscrit ancien")

display_availability(book)
display_availability(archive)

### I - Interface Segregation Principle (ISP)

**Principe :** Interfaces spécifiques plutôt qu'une interface générale.

In [ ]:
# ❌ MAUVAIS EXEMPLE
from abc import ABC, abstractmethod

class Printable(ABC):
    @abstractmethod
    def print(self):
        pass

    @abstractmethod
    def scan(self):
        pass

    @abstractmethod
    def fax(self):
        pass

class SimplePrinter(Printable):
    def print(self):
        return "Impression..."

    def scan(self):
        raise NotImplementedError("Pas de scanner!")

    def fax(self):
        raise NotImplementedError("Pas de fax!")

In [ ]:
# ✅ BON EXEMPLE
from abc import ABC, abstractmethod

class Printable(ABC):
    @abstractmethod
    def print(self):
        pass

class Scannable(ABC):
    @abstractmethod
    def scan(self):
        pass

class Faxable(ABC):
    @abstractmethod
    def fax(self):
        pass

class SimplePrinter(Printable):
    def print(self):
        return "Impression..."

class MultifunctionPrinter(Printable, Scannable, Faxable):
    def print(self):
        return "Impression..."

    def scan(self):
        return "Scan..."

    def fax(self):
        return "Fax..."

# Test
simple = SimplePrinter()
multifunction = MultifunctionPrinter()

logging.info(simple.print())
logging.info(multifunction.scan())

### D - Dependency Inversion Principle (DIP)

**Principe :** Dépendre d'abstractions, pas d'implémentations concrètes.

In [ ]:
# ❌ MAUVAIS EXEMPLE
class MySQLDatabase:
    def save(self, data):
        print("Sauvegarde dans MySQL")

class BorrowManager:
    def __init__(self):
        self.db = MySQLDatabase()  # Dépendance forte !

    def record_borrow(self, borrow):
        self.db.save(borrow)
        # Impossible de tester avec une fausse DB
        # Impossible de changer vers PostgreSQL facilement

In [ ]:
# ✅ BON EXEMPLE
from abc import ABC, abstractmethod

class DatabaseInterface(ABC):
    @abstractmethod
    def save(self, data):
        pass

class MySQLDatabase(DatabaseInterface):
    def save(self, data):
        logging.info(f"[MySQL] Sauvegarde: {data}")

class PostgreSQLDatabase(DatabaseInterface):
    def save(self, data):
        logging.info(f"[PostgreSQL] Sauvegarde: {data}")

class FakeDatabase(DatabaseInterface):
    """Pour les tests"""
    def save(self, data):
        logging.info(f"[FAKE] Sauvegarde: {data}")

class BorrowManager:
    def __init__(self, database: DatabaseInterface):
        self.db = database  # Injection de dépendance

    def record_borrow(self, borrow):
        self.db.save(borrow)

# Utilisation
db_prod = MySQLDatabase()
manager_prod = BorrowManager(db_prod)

db_test = FakeDatabase()
manager_test = BorrowManager(db_test)

# Test
from dataclasses import dataclass
from datetime import datetime

@dataclass
class Book:
    title: str

@dataclass
class Borrow:
    book: Book
    user: str

borrow = Borrow(Book("1984"), "Alice")
manager_prod.record_borrow(borrow)
manager_test.record_borrow(borrow)

## Conclusion

La POO n'est pas juste une façon d'organiser du code, c'est un outil pour :
- **Penser** en termes de responsabilités
- **Debugger** efficacement avec des logs structurés
- **Faire évoluer** le code sans tout casser
- **Collaborer** avec des interfaces claires

Les principes SOLID sont des garde-fous qui vous éviteront des heures de debugging et de refactoring.

**🎯 Conseil final :** Commencez simple, refactorisez quand la complexité augmente, et testez chaque classe individuellement.